Here I aim to tackle a very common problem faced by students.

A lot of educators are often busy with everyday life which is why they may not have time to develop questions for their students to practice daily.

Students ,too, wish to practice what they have learnt and what better way to do that than practising question?

Studies have shown that students learn better if they have been able to apply what they have beeen taught. Not to mention, most competitive exams in India are MCQ based.

**True and false options generator**

**INTUITION:**
We are using GPT-2 for sentence generation given a partial sentence for true and false statements.First import the Transformer, then take a partial sentence and use top_k and top_p sampling for most probable sentences. Therefore encoding this and generating the false statements after decoding.

In [ ]:
#importing GPT2 for sentence generation
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

GPT2tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
GPT2model = TFGPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=GPT2tokenizer.eos_token_id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
#incomplete sentence
partial_sentence = "The old woman was sitting under a tree and"
input_ids = GPT2tokenizer.encode(partial_sentence,return_tensors='tf')
print (input_ids)
maximum_length = len(partial_sentence.split())+100

tf.Tensor([[ 464 1468 2415  373 5586  739  257 5509  290]], shape=(1, 9), dtype=int32)


In [ ]:
# Activate top_k sampling and top_p sampling with only from 90% most likely words
sample_outputs = GPT2model.generate(
    input_ids,
    do_sample=True,
    max_length=maximum_length,
    top_p=0.80,# smallest set of top probable tokens whose cumulative probability is at least p
    top_k=60,  # the model considers only the top k most probable next words (or tokens)
    repetition_penalty  = 10.0,
    num_return_sequences=10 #0-9
)

In [ ]:
import nltk
nltk.download('punkt')
from nltk import tokenize
generated_sentences=[]
#generating false sentences for True and False
for i, sample_output in enumerate(sample_outputs):
    decoded_sentence = GPT2tokenizer.decode(sample_output, skip_special_tokens=True)
    # final_sentence = decoded_sentence
    final_sentence = tokenize.sent_tokenize(decoded_sentence)[0]
    generated_sentences.append(final_sentence)
    print (i,": ",final_sentence)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0 :  The old woman was sitting under a tree and saw two men in black-and white, the same one who had been talking with me as I got out of my car.
1 :  The old woman was sitting under a tree and the man behind her.
2 :  The old woman was sitting under a tree and her head in the grass.
3 :  The old woman was sitting under a tree and when the man came out, she gave him what he wanted.
4 :  The old woman was sitting under a tree and looked up.
5 :  The old woman was sitting under a tree and I walked past her.
6 :  The old woman was sitting under a tree and the young man in front of her stood behind an oak bench.
7 :  The old woman was sitting under a tree and her hair fell down from the sky like black snow.
8 :  The old woman was sitting under a tree and her legs were swaying like she didn't want to go down on them.
9 :  The old woman was sitting under a tree and the black bear came up to me, holding her face in his.


**MCQ GENERATOR**

**INTUITION:**
We are basically first extracting keywords from our data, then use Bert Summarizer for generating a summary and checking if that keyword accours in the summary. If it does,then we choose sentences based on that. After this we generate distractors for the particular keywords(here,we have only considered proper nouns as keywords but we may also select others based on convenience). Now these distractors were generated using wordnet and concept net.

*WordNet is a rich lexical resource for the English language, focusing on word meanings and relationships, while ConceptNet is a broader, multilingual semantic network designed to capture general knowledge and relationships between concepts.*

After this we use wordsense to capture meaning of a word based on context. Doing this allows us to generate better reults for distractors as the options should be in context of that paragraph and sentence.

We compile these concepts together then generate the MCQs.

In [ ]:
!pip install python-rake==1.4.4 flashtext

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for python-rake: filename=python_rake-1.4.4-py3-none-any.whl size=13437 sha256=156070d4a46a6314103ad6f4c44b6004962c545285ec61a90503784b923c0397
  Stored in directory: /root/.cache/pip/wheels/a5/8f/00/bf4e7adcb63ddd1a00f00ef3ce57ecdb35cb243f18cc894a47
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9296 sha256=c9e85d29bfed934f64224cc04a96a2a3ab678854c8309bde6bbb25fefc5a691b
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built python-rake flashtext


In [ ]:
#installing Bert Extractive Summarizer
!pip install bert-extractive-summarizer

In [ ]:
pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-9u6h8xn6
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-9u6h8xn6
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.8 MB/s eta 0:00:00
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160628 sha256=2a7e27880bb71a503f29fa874be2e44cf241fbbb208ea50864d00c9eff6c14e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-qu7n11se/wheels/8c/07/29/6b35bed2aa36e33d77ff3677eb716965ece4d2e56639ad0aab
Successfully built pke


In [ ]:
from summarizer import Summarizer

t = """Sniffer dog Tucker uses his nose to help researchers find out why a killer whale population off the northwest coast of the United States is on tKe decline. He searches for whale faeces floating on the surface of the water, which are then collected for examination. He is one of the elite team of detection dogs used by scientists studying a number of species including right whales and killer whales.

Conservation canines are fast becoming indispensable tools for biologists according to Aimee Hurt, associate director and co-founder of Working Dogs for Conservation, based in Three Forks, Montana.
Over the last few years, though, so many new conservation dog projects have sprung up that Hurt can no longer keep track of them all. Her organization’s dogs and their handlers are fully booked to assist field researchers into 2012.



“Dogs have such a phenomenal sense of smell”, explained Sam Wasser, director of the Center for Conservation biology at the University of Washington in Seattle. He has worked with scat-detection dogs since 199(g). Scientists have been using Conservation Canines in their research since 199(g). These dogs have enabled them to non-invasively access vast amount of genetic and physiological information which is used to tackle conservation problems around the world. Such information has proved vital for determining the causes and consequences of human disturbances on wildlife as well as the actions needed to mitigate such impacts.



The ideal detection dog is extremely energetic with an excessive play drive. These dogs will happily work all • day long, motivated by the expectation of a ball game as a reward for sample detection. The obsessive, high energy personalities of detection dogs also make them difficult to maintain as pets. As a result, they frequently find themselves abandoned to animal shelters, facing euthanasia. The programme rescues these dogs and offers them a satisfying career in conservation research."""
def Summary(text):
  model = Summarizer()
  result = model(text, min_length=60, max_length = 500 , ratio = 0.4)
  summarized_text = ''.join(result)
  print(summarized_text)
  return summarized_text
Summary(t)#sample

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Sniffer dog Tucker uses his nose to help researchers find out why a killer whale population off the northwest coast of the United States is on tKe decline. Over the last few years, though, so many new conservation dog projects have sprung up that Hurt can no longer keep track of them all. Scientists have been using Conservation Canines in their research since 199(g). Such information has proved vital for determining the causes and consequences of human disturbances on wildlife as well as the actions needed to mitigate such impacts. These dogs will happily work all • day long, motivated by the expectation of a ball game as a reward for sample detection. The obsessive, high energy personalities of detection dogs also make them difficult to maintain as pets.


'Sniffer dog Tucker uses his nose to help researchers find out why a killer whale population off the northwest coast of the United States is on tKe decline. Over the last few years, though, so many new conservation dog projects have sprung up that Hurt can no longer keep track of them all. Scientists have been using Conservation Canines in their research since 199(g). Such information has proved vital for determining the causes and consequences of human disturbances on wildlife as well as the actions needed to mitigate such impacts. These dogs will happily work all • day long, motivated by the expectation of a ball game as a reward for sample detection. The obsessive, high energy personalities of detection dogs also make them difficult to maintain as pets.'

In [ ]:
import string
from nltk.corpus import stopwords
import pke
import nltk
nltk.download('stopwords')
def extracting_keywords(text):
    #list to store our keywords
    keywords = []
    #initialize extractor
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(text)
    #we want to extract proper noun
    pos = {'PROPN'}

    #removing stopwords, punctuations and brackets
    stoplist = list(string.punctuation)
    stoplist+= stopwords.words('english')
    stoplist+= ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    extractor.load_document(input=text,
                        stoplist=stoplist)
    extractor.candidate_selection(pos=pos)

    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=15) #getting the n best keyphrass
    for i in keyphrases:
        keywords.append(i[0])
    return keywords

def final_keywords(text,quantity):

    keywords_from_fulltext = extracting_keywords(text)
    if(quantity=='0'):
        generated_summary = Summary(text)
        filtered_keywords = []
        for i in keywords_from_fulltext:
            if i.lower() in generated_summary.lower(): #if the keyword is important enough to be in the generated summarythen proceed
                filtered_keywords.append(i)
        print("Selected Keywords from summary :",filtered_keywords)
        return filtered_keywords,generated_summary
    else:
        print("Selected Keywords from Full Text :",keywords_from_fulltext)
        return keywords_from_fulltext,text

f = open('/content/sample_data/article.txt','r')
text = f.read()
f.close()
keywords,summary= final_keywords(text,20)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Selected Keywords from Full Text : ['conservation', 'washington', 'university', 'three forks', 'working dogs', 'center', 'seattle', 'montana', 'aimee hurt', 'sam wasser', 'whale population', 'united states', 'hurt', 'tucker', 'conservation canines']


In [ ]:
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
def set_sentances(text):
    print("Selecting Sentences based on keywords...")
    sentences = [sent_tokenize(text)]
    # nested list to single list
    sentences = [i for sent in sentences for i in sent]

    #remove short sentences
    sentences = [sent.strip() for sent in sentences if len(sent)>20]
    #print(sentences)
    return sentences


def extract_sentences(text,quantity):
    keywords,text = final_keywords(text,quantity)
    key_processor = KeywordProcessor()
    filtered_sentences = {}

    #adding keywords to processor and to dict
    for i in keywords:
        filtered_sentences[i]=[]
        key_processor.add_keyword(i)

    #calling fn to set sentences from summary text
    sentences = set_sentances(text)
    print("Filtering sentences...")
    #extracting sentences with given keywords and add to dict keys
    for sent in sentences:
        keyword_searched = key_processor.extract_keywords(sent)
        for key in keyword_searched:
            filtered_sentences[key].append(sent)
    filtered_sentences = dict([(key,val) for key,val in filtered_sentences.items() if(val)])

    #sorting with longest sentence of given keyword on top
    for i in filtered_sentences.keys():
        values = filtered_sentences[i]
        values = sorted(values,key=len,reverse=True)
        filtered_sentences[i] = values

    print(filtered_sentences)
    return filtered_sentences



In [ ]:
pip install pywsd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wn: filename=wn-0.0.23-py3-none-any.whl size=31792911 sha256=7e6f383ff23c139e7905aa374491b02eb94774ad63176c437adb9e50256e4629
  Stored in directory: /root/.cache/pip/wheels/a1/1a/7d/23a76ce45998af60e47466a694c237fa26023c5674b47672b2
Successfully built wn


In [ ]:
import requests
import re
import nltk
import pandas as pd
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
import random
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from nltk.corpus import wordnet


def wordnet_distractors(syon,word):
    print("Obtaining relative options from Wordnet...")
    distractors = []
    word = word.lower()
    ori_word = word

    #checking if word is more than one word then make it one word with _
    if len(word.split())>0:
        word = word.replace(" ","_")

    hypersyon = syon.hypernyms()
    if(len(hypersyon)==0):
        return distractors
    for i in hypersyon[0].hyponyms():
        name = i.lemmas()[0].name()

        if(name==ori_word):
            continue
        name = name.replace("_"," ")
        name = " ".join(i.capitalize() for i in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors


def conceptnet_distractors(word):
    print("Obtaining relative options from ConceptNet...")
    word = word.lower()
    orig_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = []
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term']

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and orig_word.lower() not in word2.lower():
                distractor_list.append(word2)

    return distractor_list

def word_sense(sentence,keyword):
    print("Getting word sense to obtain best MCQ options with WordNet...")
    word = keyword.lower()
    if len(word.split())>0:
        word = word.replace(" ","_")

    syon_sets = wordnet.synsets(word,'n')
    if syon_sets:
        try:
            wup = max_similarity(sentence, word, 'wup', pos='n')
            adapted_lesk_output =  adapted_lesk(sentence, word, pos='n')
            lowest_index = min(syon_sets.index(wup),syon_sets.index(adapted_lesk_output))
            return syon_sets[lowest_index]

        except:
            return syon_sets[0]

    else:
        return None


def display(text,quantity):

    filtered_sentences = extract_sentences(text,quantity)

    options_for_mcq = {}
    for keyword in filtered_sentences:
        wordsense = word_sense(filtered_sentences[keyword][0],keyword)
        if wordsense:
           distractors = wordnet_distractors(wordsense,keyword)
           if len(distractors)>0:
                options_for_mcq[keyword]=distractors
           if len(distractors)<4:
               distractors = conceptnet_distractors(keyword)
               if len(distractors)>0:
                    options_for_mcq[keyword]=distractors

        else:
            distractors = conceptnet_distractors(keyword)
            if len(distractors)>0:
                options_for_mcq[keyword] = distractors
    print(" Creating JSON response for API...")
    df = pd.DataFrame()
    cols = ['question','options','extras','answer']

    index = 1
    for i in options_for_mcq:
        sentence = filtered_sentences[i][0]
        sentence = sentence.replace("\n",'')
        pattern = re.compile(i, re.IGNORECASE)
        output = pattern.sub( " ______ ", sentence)
        print ("%s)"%(index),output)
        options = [i.capitalize()] + options_for_mcq[i]
        top4 = options[:4]
        random.shuffle(top4)
        optionsno = ['a','b','c','d']
        for idx,choice in enumerate(top4):
            print ("\t",optionsno[idx],")"," ",choice)
        print ("\nMore options: ", options[4:8],"\n\n")
        df = pd.concat([df,pd.DataFrame([[output,top4,options[4:8],i.capitalize()]],columns=cols)],axis=0)
        index = index + 1
    df.to_json('response.json',orient='records',force_ascii=False)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Warming up PyWSD (takes ~10 secs)... took 10.626807451248169 secs.


In [ ]:
with open('/content/sample_data/article.txt','r') as f:
  text = f.read()
display(text, quantity='high')

Selected Keywords from Full Text : ['conservation', 'washington', 'university', 'three forks', 'working dogs', 'center', 'seattle', 'montana', 'aimee hurt', 'sam wasser', 'whale population', 'united states', 'hurt', 'tucker', 'conservation canines']
Selecting Sentences based on keywords...
Filtering sentences...
{'conservation': ['Conservation canines are fast becoming indispensable tools for biologists according to Aimee Hurt, associate director and co-founder of Working Dogs for Conservation, based in Three Forks, Montana.', 'These dogs have enabled them to non-invasively access vast amount of genetic and physiological information which is used to tackle conservation problems around the world.', '“Dogs have such a phenomenal sense of smell”, explained Sam Wasser, director of the Center for Conservation biology at the University of Washington in Seattle.', 'Over the last few years, though, so many new conservation dog projects have sprung up that Hurt can no longer keep track of them 

Future Scope:

Had I more time I would make a video to text trancriber after which we would implement QuestionGeneration on it as for a lot of us we learn last minute through YouTube lectures.